<a href="https://colab.research.google.com/github/Akhilesh53/WebScrapping/blob/main/Beautiful%20Soup-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this file, we will extract the data present in all the pages page 1, 2, 3, 4
Also, we extract the data present in the lab name which is a link => so extract from that link too

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
lis = [dict()]

In [3]:
bad_chars = [';',',', '.', '(', ')','-']

In [4]:
for t in range(4):
    url = "https://www.ams.usda.gov/rules-regulations/hemp/dea-laboratories?field_lab_location_administrative_area=All&page="+str(t)
    req = requests.get(url)
    soup = BeautifulSoup(req.text,"html.parser")
    #print(soup.prettify())
    tb = soup.find("tbody")

    for t in tb.find_all("tr"):
      i=0;
      dic = dict()
      for d in t.find_all("td"):
        if i==0:
          dic["Lab Name"] = d.text
          for l in d.find_all("a",href=True):
            if(l.text):
              link = l['href']
              link = "https://www.ams.usda.gov/"+link
          
      
          req2 = requests.get(link)
          soup2 = BeautifulSoup(req2.text,"html.parser")
          #print("prettify")
      
          lc =soup2.find("div",attrs={"class":"field field--name-field-lab-location field--type-address field--label-above"})
          address = lc.find("span",attrs={"class":"address-line1"})
          #print(address.text)

          locality =lc.find("span",attrs={"class":"locality"})
          #print(locality.text)

          admin =lc.find("span",attrs={"class":"administrative-area"})
          #print(admin.text)

          postal =lc.find("span",attrs={"class":"postal-code"})
          #print(postal.text)

          country =lc.find("span",attrs={"class":"country"})
          #print(country.text)

          dic["Lab Location"] = address.text + " " + locality.text + " " + admin.text + " " + postal.text + " " + country.text 
      
          if(soup2.find("div",attrs={"class":"field field--name-field-phone-number field--type-string field--label-above"})):
            dic["Phone Number"] = soup2.find("div",attrs={"class":"field field--name-field-phone-number field--type-string field--label-above"}).find("div",attrs={"class":"field__item"}).text
          else:
            dic["Phone Number"]= "No Content"
          
          if(soup2.find("div",attrs={"class":"field field--name-field-contact field--type-string field--label-above"})):
            dic["Contact"] = soup2.find("div",attrs={"class":"field field--name-field-contact field--type-string field--label-above"}).find("div",attrs={"class":"field__item"}).text
          else:
            dic["Contact"] = "No content"

          if(soup2.find("div",attrs={"class":"field field--name-field-contact-email field--type-email field--label-above"})):
            dic["Email"] = soup2.find("div",attrs={"class":"field field--name-field-contact-email field--type-email field--label-above"}).find("div",attrs={"class":"field__item"}).text
          else:
            dic["Email"] = "No Content"

    
          #print(link)
          i =(i+1)%3
        elif i==1:
          #print(d.text)
          dic["City"] = d.text

      lis.append(dic)


In [6]:
del lis[0:1]

In [7]:
print(len(lis))

78


In [9]:
df = pd.DataFrame(lis)
df

,Lab Name,Lab Location,Phone Number,Contact,Email,City
0,"Accelerated Analytical Labs, Inc.",9075 W. Heather Ave. Milwaukee WI 53224 United...,(414) 409-3887,Paul Lillyroot,plillyroot@acceleratedlabs.com,Wisconsin
1,Accelerated Cannabis Testing (ACT),9075 W. Heather Ave. Milwaukee WI 53224 United...,(414) 409-3887,Paul Lillyroot,plillyroot@acceleratedlabs.com,Wisconsin
2,AccuScience Laboratories,40 S Dewey Street Eustis FL 34726 United States,407-342-5755,"Dr. Harry Behzadi, Ph.D., President/CEO",hbehzadi@accusciencelabs.com,Florida
3,ACS Laboratory LLC,721 Cortaro Drive Sun City Center FL 33573 Un...,8136344529 ext.317,Yasmin Lee,ylee@acslabcannabis.com,Florida
4,Agrozen Laboratory,417 Ransdell Road Lebanon IN 46052 United States,844-655-6935,"Brian Schroeder, CEO",lab@agrozen.com,Indiana
...,...,...,...,...,...,...
73,"Universal Diagnostics, LLC",673 North Bardstown Road Mt. Washington KY 400...,(502) 444-2044,Daniel Burriss,d.burriss@ud-labs.com,Kentucky
74,"US Pharma, Ltd.",13900 NW 57th Court Miami Lakes FL 33014 Uni...,"(305) 698-4600, extension 4711",Nahomie Myrtil,nahomie.myrtil@uspharmaltd.com,Florida
75,"Waters Agricultural Laboratories, Inc.",257 Newton Road Camilla GA 31730 United States,"229-336-7216, Fax: 229-336-0977","Keith Dominey, CEO",info@watersag.com,Georgia
76,"XenoBiotic Laboratories, Inc. | WuXi AppTec",107 Morgan Lane Plainsboro NJ 08536 United States,(609) 606-6620,"Dennis Heller, Ph.D.",dennis_heller@wuxiapptec.com,New Jersey
